#### This notebook contains code used to populate data dictionary
**Caution Note:** whenever an excel write function is executed the file is overwritten!

It is done step by step with different chunks of calls.

It is in development state and not cleaned at all!

In [1]:
import pandas as pd
from pandas_datareader import wb
from data_dictionary.append_df_to_excel import append_df_to_excel

C:\Users\beto\anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# read one sheet from excel tmee database provided by Eduard (and modified by Beto)
path_read_file = './populate_data_dictionary/TM Indicators Flow Revised Database - 2020306 SP ED ECD -working file v-6 Aug 2020_beto.xlsx'
indicators = pd.read_excel(path_read_file, sheet_name='TM Revised database', header = 2)

#### Indicators New or Retained

In [3]:
# filter indicators that are new or retained from excel tmee data base
logic_not_removed = indicators['Status in the new `db'].str.lower().str.contains('new|retained')
# filter logic_not_removed that are not boolean
logic_not_null = logic_not_removed.notnull()
# operator AND for two logics above
logic_not_removed_null = logic_not_removed & logic_not_null
not_removed_indicators = indicators[logic_not_removed_null]

#### Indicators specified with Unesco Institute for Statistics (UIS) API source address

In [4]:
# point to indicators that are extracted from UIS
logic_UIS = not_removed_indicators['Data Source'].str.lower().str.contains("api.uis.unesco.org")
# filter not boolean from logic_UIS
logic_UIS_not_null = logic_UIS.notnull()
# operator AND for two logics above
logic_UIS_not_null = logic_UIS & logic_UIS_not_null
print(f"There are {logic_UIS_not_null.sum()} indicators extracted from UIS.")

There are 129 indicators extracted from UIS.


In [5]:
# create new column in dataframe with their corresponding codes
not_removed_indicators['Code'] = not_removed_indicators['Indicator Code'][logic_UIS_not_null].str.replace(' ','_')

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Indicators specified with API source in Helix

In [6]:
# point to indicators with helix: or helix code: in excel tmee data base
logic_helix = not_removed_indicators['Data Source'].str.lower().str.contains("helix")
# filter logic_helix that are not boolean
logic_helix_not_null = logic_helix.notnull()
# operator AND for two logics above
logic_helix_not_null = logic_helix & logic_helix_not_null
print(f"There are {logic_helix_not_null.sum()} indicators extracted from Helix.")

There are 33 indicators extracted from Helix.


In [7]:
# populate dataframe with the corresponding codes from Helix Source
# before: eliminate all Helix ... prefix
not_removed_indicators['Code'][logic_helix_not_null] = not_removed_indicators[logic_helix_not_null]['Data Source']\
.str.replace(r'^.*?: ', '', regex=True)\
.str.replace('/', '').str.replace('\\', '')
# consider something more elegant with regex for the line above?
# consider something stronger with respect to blank spaces in the regex line!

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\beto\anaconda3\lib\site-packages\pandas\core\generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#### Write Source names for Helix & UIS

In [8]:
# create new column in dataframe for the corresponding Source Names
# standardize helix Api source names adding 'Helix:' prefix to code names
not_removed_indicators['Source Name'] = 'Helix: ' + not_removed_indicators[logic_helix_not_null]['Code']

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# standardize UIS Api source names adding 'UIS:' prefix to code names
not_removed_indicators['Source Name'][logic_UIS_not_null] = 'UIS: ' + not_removed_indicators[logic_UIS_not_null]['Code']

C:\Users\beto\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Get Indicators Addresses, retrieved by UIS API

In [10]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
# Keep from sources only the address 'root' before country specification
sources_uis = sources_uis.str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# note: the logic above doesn't work when indicator data source cell contains a new line before the address

#### Get Indicators retrieved by NSI

In [11]:
# point to indicators that contains NSI in Data Source column (excel tmee data base)
logic_nsi = not_removed_indicators['Data Source'].str.lower().str.contains("nsi")
# filter logic_nsi that are not boolean
logic_nsi_not_null = logic_nsi.notnull()
# operator AND for two logics above
logic_nsi_not_null = logic_nsi & logic_nsi_not_null
print(f"There are {logic_nsi_not_null.sum()} indicators extracted from NSI.")

There are 233 indicators extracted from NSI.


#### Research
##### Adding more possible Helix!
There are three marked as 'Helix' only, plus: `ECD_CHLD_U5_BKS-HM`, `ECD_CHLD_U5_PLYTH-HM` and 'SOWC 2019'
* Marked as 'Helix' only worked fine, end up being Unicef WS indicators (`WS_SCH_H-B`, `WS_SCH_S-B` and `WS_SCH_W-B`).
* Indicators `ECD_CHLD_U5_BKS-HM` and `ECD_CHLD_U5_PLYTH-HM` also work fine!
* SOWC 2019 haven't found (net migration rate 2015-2020, supposed to be calculated by UNICEF)

In [ ]:
logic_more_helix = not_removed_indicators['Data Source'].str.lower().str.contains("sowc 2019")
logic_more_helix = logic_more_helix & logic_more_helix.notnull()
not_removed_indicators[logic_more_helix]

##### World Bank
Those that are already there, I will add them with pd data reader and the code.

In [ ]:
indicator_name = 'LO.PISA.MAT'
pisa_mat = wb.download(indicator=indicator_name, country=['AL'], start=1950, end=2020)
indicator_name = 'LO.PISA.MAT.FE'
pisa_mat_fe = wb.download(indicator=indicator_name, country=['AL'], start=1970, end=2020)
indicator_name = 'LO.PISA.MAT.MA'
pisa_mat_ma = wb.download(indicator=indicator_name, country=['AL'], start=1970, end=2020)

In [ ]:
indicator_name = 'LO.PISA.MAT'
pisa_mat = wb.download(indicator=indicator_name, start=1970, end=2070)
indicator_name = 'LO.PISA.MAT.FE'
pisa_mat_fe = wb.download(indicator=indicator_name, start=1970, end=2070)
indicator_name = 'LO.PISA.MAT.MA'
pisa_mat_ma = wb.download(indicator=indicator_name, start=1970, end=2070)

In [ ]:
pd_all = pd.concat([pisa_mat,pisa_mat_fe,pisa_mat_ma])
pd_all

#### Working out what remains to be Extracted!
So far we have API's UIS and Helix

In [ ]:
logic_not_helix_uis = ~(logic_helix_not_null | logic_UIS_not_null)

In [ ]:
# this is basically the info to process
not_removed_indicators['Data Source'][logic_not_helix_uis].unique()

In [ ]:
# indicators with no info
not_removed_indicators['Data Source'][logic_not_helix_uis].isna().sum()

In [ ]:
# indicators extracted from NSI
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("nsi").sum()

In [ ]:
# indicators extracted from calculations
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("calculated").sum()

In [ ]:
# indicators from 'NSI or/and EuroStat'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains(" eurostat").sum())
# indicators from 'Eurostat & WB'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("eurostat ").sum())
# indicators from 'Eurostat'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("eurostat").sum())

In [ ]:
# indicators extracted from WB or 'Eurostat & WB'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("wb").sum())
# indicators with address containing 'worldbank'
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("worldbank").sum())
# indicators marked as data reader comes from world bank
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("data reader").sum())

In [ ]:
# contains just 'WB' or 'Eurostat & WB' and nothing else (no source specified)
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'WB').sum())
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'Eurostat & WB').sum())

In [ ]:
# contains api in the address (these are all world bank API)
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("api").sum())
print(f"From WB only {round(11/57*100,2)}% specifies API")

In [ ]:
# indicators extracted from WHO
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("who").sum()

In [ ]:
# indicators extracted from ILO
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("ilo").sum()

In [ ]:
# contains just 'ILO' and nothing else (no source specified)
print((not_removed_indicators['Data Source'][logic_not_helix_uis] == 'ILO').sum())
print(f"From ILO only {round(1/41*100,2)}% specifies API")

In [ ]:
# indicators to search in Helix
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("helix").sum()

In [ ]:
# indicators to search in UIS
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("uis").sum()

In [ ]:
# indicators marked with MICS
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("mics").sum()

In [ ]:
# indicators marked with SMQs
not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("smq").sum()

In [ ]:
# indicators to search in TM
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("tm").sum())
# indicators from web not scrapable
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("not scrap").sum())
# indicators from web not scrapable
print(not_removed_indicators['Data Source'][logic_not_helix_uis].str.lower().str.contains("ptb").sum())

#### Appends to Data Dictionary
**Caution Note:** whenever an excel write function is executed the file is overwritten!
##### Append info Indicators Code - API's from UIS & Helix

In [ ]:
# path to file for appending
path_write_file = './populate_data_dictionary/indicator_dictionary_TM_v2.xlsx'

In [ ]:
# use a function from stackoverflow to append data dictionary
append_df_to_excel(path_write_file, not_removed_indicators.iloc[21:,[105]], sheet_name='Indicator', startrow=22,
                   startcol = 5, header = False, index = False)

##### Append info Indicators Source Name - API's from UIS & Helix

In [ ]:
# place list of helix and UIS api's in source
logic_helix_uis = not_removed_indicators['Source Name'].notna()
append_df_to_excel(path_write_file, not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:],
                   sheet_name='Source', startrow=6,
                   startcol = 2, header = False, index = False)

##### Append info Indicators Addresses - UIS API

In [ ]:
# place list UIS api's addresses in source spreadsheet
append_df_to_excel(path_write_file, sources_uis,
                   sheet_name='Source', startrow=26,
                   startcol = 3, header = False, index = False)

#### Dev testing zone

In [ ]:
sources_uis

In [ ]:
not_removed_indicators.loc[logic_helix_uis,'Source Name'].iloc[2:]

In [ ]:
not_removed_indicators.iloc[105]

In [ ]:
# create new column in dataframe with codes with add
# get indicator names

In [ ]:
# Keep from sources only the address 'root' before country specification
sources_uis[logic_api].str.replace(r'^.*?https', 'https', regex=True).apply(lambda x: x.split('AL+')[0])
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'
# 'https://api.uis.unesco.org/sdmx/data/UNESCO,SDG4,2.0/PQTR..L3................'

In [ ]:
sources_uis = not_removed_indicators['Data Source'][logic_UIS_not_null]
logic_api = sources_uis.str.lower().str.contains("api.uis.unesco.org")

In [ ]:
# not_removed_indicators['Code'] = not_removed_indicators[logic_UIS_not_null]['Data Source']